## Исследование надёжности заёмщиков

Исследования для кредитного отдела банка.

Необходимо определить какие параметры влияют на факт погашения кредита клиентом в срок.


### Описание данных:
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита


### План выполнения проекта:
    Шаг 1. Открыть таблицу и изучить данные
    Шаг 2. Предобработка данных
        2.1 определите и заполните пропущенные значения;
        2.2 замените вещественный тип данных на целочисленный;
        2.3 удалите дубликаты;
        2.4 выделите леммы в значениях столбца с целями получения кредита;
        2.5 категоризируйте данные.
    Шаг 3. Ответить на вопросы:
        Есть ли зависимость между наличием детей и возвратом кредита в срок?
        Есть ли зависимость между семейным положением и возвратом кредита в срок?
        Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
        Как разные цели кредита влияют на его возврат в срок?


### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# Импорт библиотек
import pandas as pd
import IPython.display
from pymystem3 import Mystem
from collections import Counter

In [2]:
# Выгрузка данных и анализ
data = pd.read_csv('**')
data.info()
data.columns
display(data.isnull().sum())
display(data.duplicated().sum())
display(data.head()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

54

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В качестве данных была дана таблица с 12-ю столбцами и 21525 строками. Присутствуют категориальные и количественные данные о людях. Два столбца с пропусками: 'days_employed' и 'total_income'. Присутвует 54 дубликата. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
print(data.isna().sum())
# убираем отрицательные значения
data['days_employed'] = data['days_employed'].apply(abs)
data['dob_years'] = data['dob_years'].apply(abs)
data['children'] = data['children'].apply(abs)
data['total_income'] = data['total_income'].apply(abs)


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [4]:
# заполняем пропуски медианными значениями
for dob_years in data['dob_years'].unique():
    median = data.loc[data['dob_years'] == dob_years, 'days_employed'].median()
    #print(median, dob_years) # дополнила код выводом
    data.loc[(data['dob_years'] == dob_years) & (data['days_employed'].isna()), 'days_employed'] = median

for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(median, income_type)  # дополнила код выводом
    data.loc[(data['income_type'] == income_type) & (data['total_income'].isna()), 'total_income'] = median

data.isnull().sum()

142594.39684740017 сотрудник
118514.48641164352 пенсионер
172357.95096577113 компаньон
150447.9352830068 госслужащий
131339.7516762103 безработный
499163.1449470857 предприниматель
98201.62531401133 студент
53829.13072905995 в декрете


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Пропуски присутсвовали в столбцах с количественными переменными: "общий трудовой стаж в днях" и "ежемесячный доход". 
Возможные причины появления пропусков:
- клиенты не знали(не помнили) свой трудовой стаж при заполнении документов;
- клиенты не имели фиксированного дохода;
- не было указано за какой срок нужен ежемесячный доход;
- клиенты поленились посчитать рабочие дни.

Перед заполнением пропусков были взяты модули всех количественных значений. Они не могут быть по природе своей отрицательными, следовательно, "-" - это опечатка.

Заполнили пропуски в столбце days_employed с группировкой по переменной dob_years, так как, зачастую, трудовой стаж напрямую зависит от возраста клиента. 

Заполнили пропуски в столбце total_income с группировкой по переменной income_type, так как, ежемесячный доход в зависимости от типа занятости схож.

### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

### Обработка дубликатов

In [6]:
print(data['family_status'].value_counts())  
print(data['gender'].value_counts())
print(data['income_type'].value_counts())
print(data['education'].value_counts())  



женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [7]:
# приводим данные к нижнему регистру
data['education'] = data['education'].str.lower() 
# Обрабатываем дубликаты
print('Количество дубликатов до',data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True) 
print('Количество дубликатов после',data.duplicated().sum())

data['children'].value_counts()
data.loc[data['children'] == 20, 'children'] = 2


Количество дубликатов до 71
Количество дубликатов после 0


### Вывод

Дубликаты искали методом value_counts(), применяя к различным столбцам. 

В столбце education были найдены дубликаты с разными регистрами. Для их исключения привели весь столбец к нижнему регистру. Подобные дубликаты появляются из-за того, что недостаточно конкретизации в бланках для заполнения или из-за автозамены на телефонах.

В столбце children обнаружили нестандартное значение - 20. Его заменили на 2, так как это, вероятнее всего, опечатка.

### Лемматизация

In [8]:
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [9]:
m = Mystem()

data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize)
print(data['purpose_lemmas'].value_counts())    # вывод данных для анализа часто появляющихся причин кредитов
Counter(['purpose_lemmas'])
# создание функции, которая по смыслу классифицирует причины кредитов 
def sum_purpose (lemma):
    if 'жилье' in lemma or 'недвижимость' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль' 
    if 'образование'in lemma:
        return 'образование' 
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'ремонт' in lemma:
        return 'ремонт'
    if 'коммерческий' in lemma:
        return 'коммерция'
    return 'другое'

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [10]:
data['purpose_short'] = data['purpose_lemmas'].apply(sum_purpose) 

### Вывод

Лемматизацию проводили с использованием библиотек pymystem3 и collections. Оттуда мы применяли методы для приведения слов к словарному виду и плдсчёта одинаковых слов. 

Далее была написана функция, которая позволяла в соответствии с леммами разделить цели кредитов на несколько категорий: недвижимость, автомобиль, образование, свадьба, ремонт и коммерческие траты.

### Категоризация данных

In [11]:
data_dict_edu = data[['education_id', 'education' ]]
data_dict_edu = data_dict_edu.drop_duplicates().reset_index(drop = True)
data_dict_stat = data[['family_status_id', 'family_status' ]]
data_dict_stat = data_dict_stat.drop_duplicates().reset_index(drop = True)

In [12]:
# категоризация по возрасту
def age_group(age):
    if age < 18:
        return 'дети'
    if age <= 64:
         return 'взрослые'
    return 'пенсионеры'
data['age_group'] = data['dob_years'].apply(age_group)
print(data['age_group'].value_counts())

взрослые      20458
пенсионеры      895
дети            101
Name: age_group, dtype: int64


In [13]:
#data.drop(data[data['age_group'] == 'дети'].index, inplace = True)  # выявление нестандартных данных и замена медианными значениями
# заполнение пропусков в доходах клиентов медианным значением по возрастным группам
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'dob_years'].median()
    print(median, income_type)  # дополнила код выводом
    data.loc[(data['income_type'] == income_type) & (data['dob_years'] == 0), 'dob_years'] = median
print(data['age_group'].value_counts())

39.0 сотрудник
60.0 пенсионер
39.0 компаньон
40.0 госслужащий
38.0 безработный
42.5 предприниматель
22.0 студент
39.0 в декрете
взрослые      20458
пенсионеры      895
дети            101
Name: age_group, dtype: int64


In [14]:
# категоризация по доходу
def income_group(income):
    if income < 50000:
        return 'низкий доход'
    if 50000 <= income < 800000 :
         return 'средний доход'
    return 'высокий доход'
data['income_group'] = data['total_income'].apply(income_group)

In [15]:
print (data.groupby('education')['education'].count())

education
высшее                  5250
начальное                282
неоконченное высшее      744
среднее                15172
ученая степень             6
Name: education, dtype: int64


### Вывод

Было выделено два "словаря": образование и семейный статус. Они являются справочнками о том, какому id  какое название соотвествует. Такие словари позволят уменьшить размер таблицы, но не усложнят её использование.

Написано две функции, которые позволяют разбить данные на категории. Первая разбивает на возрастные группы, вторая - на группы по доходам.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
# создала сводную таблицу, в которой считается количество должников и их процент. Данные сгрупированы по категории 'children'
data_pivot = data.pivot_table(index = ['children'], columns = 'debt' , values = 'total_income', aggfunc = 'count')
data_pivot.columns = ['no_debt', 'debt']
data_pivot['%'] = data_pivot['debt'] / (data_pivot['debt'] + data_pivot['no_debt']) #исправила вычисления
data_pivot['quantity'] = data['children'].value_counts()
data_pivot.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})


,no_debt,debt,%,quantity
children,,,,
0,13028,1063,7.54%,14091
1,4410,445,9.17%,4855
2,1926,202,9.49%,2128
3,303,27,8.18%,330
4,37,4,9.76%,41
5,9,nan,nan%,9


### Вывод

Судя по полученным данным, нет прямой зависимости между количеством детей и возвращением кредитов в срок. Но можно заметить, что семьи, где много детей достаточно порядочные кредиторы. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
# создала сводную таблицу, в которой считается количество должников и их процент. Данные сгрупированы по категории 'family_status'
data_pivot_1 = data.pivot_table(index = ['family_status'], columns = 'debt' , values = 'total_income', aggfunc = 'count')
data_pivot_1.columns = ['no_debt', 'debt']
data_pivot_1['%'] = data_pivot_1['debt'] / (data_pivot_1['debt'] + data_pivot_1['no_debt']) #исправила вычисления
data_pivot_1['quantity'] = data['family_status'].value_counts()
data_pivot_1.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})


,no_debt,debt,%,quantity
family_status,,,,
Не женат / не замужем,2536,274,9.75%,2810
в разводе,1110,85,7.11%,1195
вдовец / вдова,896,63,6.57%,959
гражданский брак,3763,388,9.35%,4151
женат / замужем,11408,931,7.55%,12339


### Вывод

Возраст вдовцов/вдов в среднем больше, как и их способность врзвращать кредиты. Люди, которые состоят в браке или разведены, более ответственны в плане возврата кредитов. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
# создала сводную таблицу, в которой считается количество должников и их процент. Данные сгрупированы по категории 'income_group'
data_pivot_2 = data.pivot_table(index = ['income_group'], columns = 'debt' , values = 'total_income', aggfunc = 'count')
data_pivot_2.columns = ['no_debt', 'debt']
data_pivot_2['%'] = data_pivot_2['debt'] / (data_pivot_2['debt'] + data_pivot_2['no_debt']) #исправила вычисления
data_pivot_2['quantity'] = data['income_group'].value_counts()
data_pivot_2.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%,quantity
income_group,,,,
высокий доход,39,4,9.30%,43
низкий доход,349,23,6.18%,372
средний доход,19325,1714,8.15%,21039


### Вывод

Люди с низким доходом возвращают кредиты в срок значительно лучше, чем люди с большим достатком.

<div class="alert alert-info" role="alert">
  Клиентов со средний доходом чаще всего берут кредиты. Но лучше всего их вохвращают люди с  низким достатком. Возможно, они внимательнее относятся к своим финансам.
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
# создала сводную таблицу, в которой считается количество должников и их процент. Данные сгрупированы по категории 'purpose'
data_pivot_3 = data.pivot_table(index = ['purpose_short'], columns = 'debt' , values = 'total_income', aggfunc = 'count')
data_pivot_3.columns = ['no_debt', 'debt']
data_pivot_3['%'] = data_pivot_3['debt'] / (data_pivot_3['debt'] + data_pivot_3['no_debt']) #исправила вычисления
data_pivot_3['quantity'] = data['purpose_short'].value_counts()
data_pivot_3.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%,quantity
purpose_short,,,,
автомобиль,3903,403,9.36%,4306
недвижимость,10029,782,7.23%,10811
образование,3643,370,9.22%,4013
свадьба,2138,186,8.00%,2324


### Вывод

Клиенты, которые хотят купить недвижимость, лучше следят за своей кредитной историей. 

## В ходе исследования были сделаны следующие выводы: 
    - клиенты без детей являются менее ответственными заёмщиками; 
    - клиенты, состоящие в официальном браке хуже возвращают кредиты; 
    - кредиты на свадьбу вернутся, с большей вероятность, вовремя, чем кредит на недвижимость; 
    - люди со средним образованием чаще других обращаются за кредитоми в банк.
    
    
    